In [26]:
import logic
import logic2

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait

import random
import numpy as np

import os
import time

In [27]:
import numpy as np
from enum import Enum

class Direction(Enum):
    UP = 0
    DOWN = 1
    LEFT = 2
    RIGHT = 3

class Node:
    def __init__(self, state, parent=None, move=None):
        self.state = state
        self.parent = parent
        self.move = move

def move_board(board, direction):
    new_board = np.copy(board)
    moved = False
    
    if direction == Direction.UP or direction == Direction.DOWN:
        new_board = new_board.T
    
    if direction == Direction.DOWN or direction == Direction.RIGHT:
        new_board = np.flip(new_board, axis=1)

    for row in range(new_board.shape[0]):
        non_zeros = new_board[row, new_board[row] != 0]
        merged = []

        i = 0
        while i < len(non_zeros):
            if i < len(non_zeros) - 1 and non_zeros[i] == non_zeros[i + 1]:
                merged.append(2 * non_zeros[i])
                i += 1
            else:
                merged.append(non_zeros[i])
            i += 1

        while len(merged) < new_board.shape[1]:
            merged.append(0)

        if not np.array_equal(new_board[row], merged):
            moved = True

        new_board[row] = merged

    if direction == Direction.DOWN or direction == Direction.RIGHT:
        new_board = np.flip(new_board, axis=1)

    if direction == Direction.UP or direction == Direction.DOWN:
        new_board = new_board.T

    return moved, new_board

def expand_node(node):
    children = []
    
    for direction in Direction:
        moved, new_board = move_board(node.state, direction)
        
        if moved:
            child = Node(new_board, parent=node, move=direction)
            children.append(child)
    
    return children

def heuristic(state):
    return (np.max(state)*2) + (np.sum(board == 0)**2) + np.sum(state)

def dls(current_node, depth):
    if depth == 0:
        return heuristic(current_node.state), current_node.move

    best_score = -1
    best_move = None

    if depth > 0:
        for child in expand_node(current_node):
            score, move = dls(child, depth - 1)
            
            if score > best_score:
                best_score = score
                best_move = move if current_node.move is None else current_node.move

    return best_score, best_move

def iddfs(board, max_depth):
    root = Node(board)
    best_move = None
    best_score = -1
    
    for depth in range(max_depth):
        score, move = dls(root, depth)
        
        if score > best_score:
            best_score = score
            best_move = move
    
    return best_move

In [28]:
import numpy as np
import random
from copy import deepcopy
import concurrent.futures

def start_2048():
    board = np.zeros((4, 4), dtype=int)

    for _ in range(2):
        spawn_tile(board)

    return board

def spawn_tile(board, tile_value=None):
    empty_cells = list(zip(*np.where(board == 0)))
    if not empty_cells:
        return board
    
    if tile_value is None:
        tile_value = 2 if random.random() < 0.9 else 4

    row, col = random.choice(empty_cells)
    board[row, col] = tile_value
    return board

def move_board(board, move):
    moved = False

    if move in ["up", "down"]:
        board = board.T

    for row in range(4):
        if move in ["right", "down"]:
            board[row] = board[row][::-1]

        moved_row, row_changed = move_row(board[row])
        board[row] = moved_row

        if not moved and row_changed:
            moved = True

        if move in ["right", "down"]:
            board[row] = board[row][::-1]

    if move in ["up", "down"]:
        board = board.T

    return board, moved

def move_row(row):
    non_zeros = row[row != 0]
    new_row = np.zeros_like(row)

    i = 0
    skip = False
    for j in range(len(non_zeros)):
        if skip:
            skip = False
            continue

        if j < len(non_zeros) - 1 and non_zeros[j] == non_zeros[j + 1]:
            new_row[i] = non_zeros[j] * 2
            skip = True
        else:
            new_row[i] = non_zeros[j]

        i += 1

    return new_row, not np.array_equal(new_row, row)

def is_game_over(board):
    if not np.any(board == 0):
        for move in ["up", "down", "left", "right"]:
            temp_board, moved = move_board(board.copy(), move)
            if moved:
                return False
        return True
    return False

def make_move(board, move):
    if move not in ["up", "down", "left", "right"]:
        print("Invalid move. Please enter up, down, left, or right.")
        return None

    new_board, moved = move_board(board.copy(), move)

    if not moved:
        print("Invalid move. No change in board.")
        return None

    spawn_tile(new_board)
    game_over = is_game_over(new_board)

    if game_over:
        print("Game over! Final board:")
        print(new_board)
        return None

    return new_board

def get_corner_values(array):
    if array.ndim != 2:
        raise ValueError("Only 2D arrays are supported.")
    
    rows, cols = array.shape
    top_left = array[0, 0]
    top_right = array[0, cols-1]
    bottom_left = array[rows-1, 0]
    bottom_right = array[rows-1, cols-1]
    
    return np.array([top_left, top_right, bottom_left, bottom_right]).max()

def score_board(board):
    # A simple heuristic to score the board based on the highest tile and number of empty cells
    highest_tile = np.max(board)
    empty_cells = np.sum(board == 0)
    int_board = np.array(board)
    
    if (np.max(board) == get_corner_values(int_board)):
        extra_value = int_board.max() * 5
    else:
        extra_value = -int_board.max()
    
    return highest_tile + (empty_cells**3) + extra_value

def minimax(board, depth, maximizing_player, alpha, beta):
    if depth == 0 or is_game_over(board):
        return score_board(board)

    if maximizing_player:
        max_value = float('-inf')
        for move in ["up", "down", "left", "right"]:
            new_board, moved = move_board(deepcopy(board), move)
            if moved:
                value = minimax(new_board, depth - 1, False, alpha, beta)
                max_value = max(max_value, value)
                alpha = max(alpha, max_value)
                if beta <= alpha:
                    break
        return max_value
    else:
        min_value = float('inf')
        new_boards = [spawn_tile(deepcopy(board), tile_value=2), spawn_tile(deepcopy(board), tile_value=4)]
        for new_board in new_boards:
            value = minimax(new_board, depth - 1, True, alpha, beta)
            min_value = min(min_value, value)
            beta = min(beta, min_value)
            if beta <= alpha:
                break
        return min_value

def best_move_helper(args):
    move, board, depth = args
    new_board, moved = move_board(deepcopy(board), move)
    if moved:
        value = minimax(new_board, depth - 1, False, float('-inf'), float('inf'))
        return value, move
    return float('-inf'), None

def best_move(board, depth):
    with concurrent.futures.ThreadPoolExecutor() as executor:
        moves = ["up", "down", "left", "right"]
        args = [(move, board, depth) for move in moves]
        results = list(executor.map(best_move_helper, args))

    best_value, best_move = max(results, key=lambda x: x[0])
    return best_move

In [31]:
def juego(depth): #funcion que juega 2048
    current_username = os.getlogin() #obtiene el nombre de usuario de la computadora

    driver_path_mac = "Users/davidesquer/Documents/chromedriverfolder/chromedrive"  #direccion del driver de chrome
    driver_path_windows = "C:/Users/David/Downloads/chromedriver_win32/chromedriver.exe" #direccion del driver de chrome

    if current_username == "David": #checa si el nombre de usuario es David
        driver_path = driver_path_windows #si es David, usa la direccion del driver de chrome para windows
    else:
        driver_path = driver_path_mac #si no es David, usa la direccion del driver de chrome para mac

    driver = webdriver.Chrome(executable_path=driver_path) #abre el navegador
    driver.get("https://2048.io") #abre el juego

    moves = {"up": Keys.UP, "right": Keys.RIGHT, "down": Keys.DOWN, "left": Keys.LEFT} #diccionario de movimientos posibles
    moves2 = {Direction.UP: Keys.UP, Direction.RIGHT: Keys.RIGHT, Direction.DOWN: Keys.DOWN, Direction.LEFT: Keys.LEFT} #diccionario de movimientos posibles

    while True: #loop infinito, hasta que el juego termine
        l1 = []
        js_code = """
        var elements = document.querySelectorAll(".tile");
        var classNames = [];
        for (var i = 0; i < elements.length; i++) {
            classNames.push(elements[i].className);
        }
        return classNames;
        """
        class_names = driver.execute_script(js_code)
        for i in class_names:
            l1.append(i.split(" "))
            
        board = np.array([[0,0,0,0], [0,0,0,0], [0,0,0,0], [0,0,0,0]]) #matriz que contiene el estado del juego actual en ceros
        for i in range(len(l1)): #loop sobre el estado del juego actual
            value = int(l1[i][1].split("-")[1]) #valor de la casilla

            col = int(l1[i][2].split("-")[2])-1 #columna de la casilla
            row = int(l1[i][2].split("-")[3])-1 #fila de la casilla

            if board[row][col] < value: #si el valor de la casilla es mayor al valor de la casilla en la matriz, se actualiza el valor de la casilla en la matriz
                board[row][col] = value #actualiza el valor de la casilla en la matriz
        
        elements = driver.find_elements(By.CSS_SELECTOR, ".game-message") #elemento de mensaje de juego

        if elements[0].text == "Game over!\nTry again": #si el juego termina, se rompe el loop
            break
        else:
            best_move2 = iddfs(board, depth)
            if best_move2 == None:
                move = best_move(board, depth) #mejor movimiento que se puede realizar
                game_element = driver.find_element(By.TAG_NAME, "body") #elemento para mandar los movimientos
                game_element.send_keys(moves[move]) #ejecuta el movimiento elegido
            else:
                game_element = driver.find_element(By.TAG_NAME, "body") #elemento para mandar los movimientos
                game_element.send_keys(moves2[best_move2]) #ejecuta el movimiento elegido
        
    driver.close() #cierra el navegador

In [32]:
juego(5)

/var/folders/2g/ct_05dv51vbb20ykfhzzwxxm0000gn/T/ipykernel_45583/265746227.py:12: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path=driver_path) #abre el navegador


ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()